In [ ]:
import sympy.physics.mechanics as me
import sympy as sm
from scipy.integrate import odeint, solve_ivp
from scipy.interpolate import pchip
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation
from IPython.display import HTML
import matplotlib
import time
matplotlib.rcParams['animation.embed_limit'] = 2**128

This is to model **Newton's Pendulum**. If the leftmost pedulum
is moved out and released, it hits the pendulums resting, and only the rightmost one should swing, and
back again.

The pendulums which are hit do not remain totally motionless as the link below shows.

https://en.wikipedia.org/wiki/File:Newton%E2%80%99s_cradle_slo_mo.webm


**Note about the force during the collisions**

 **Hunt Crossley's method**
 
My reference is this article, given to me by JM\
https://www.sciencedirect.com/science/article/pii/S0094114X23000782 \

 
This is with dissipation during the collision, the general force is given in (63) as\
$f_n = k_0 \cdot \rho + \chi \cdot \dot \rho$, with $k_0$ as above, $\rho$ the penetration, and $\dot\rho$ the speed of the penetration.\
In the article it is stated, that $n = \frac{3}{2}$ is a good choice, it is derived in Hertz' approach. Of course, $\rho, \dot\rho$ must be the magnitudes of the respective vectors.

A more realistic force is given in (64) as:\
$f_n = k_0 \cdot \rho^n + \chi \cdot \rho^n\cdot \dot \rho$, as this avoids discontinuity at the moment of impact.

**Hunt and Crossley** give this value for $\chi$, see table 1:

$\chi = \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{k_0}{\dot \rho^{(-)}}$, 
where $c_\tau = \dfrac{v_1^{(+)} - v_2^{(+)}}{v_1^{(-)} - v_2^{(-)}}$, where $v_i^{(-)}, v_i^{(+)}$ are the speeds of $body_i$, before and after the collosion, see (45), $\dot\rho^{(-)}$ is the speed right at the time the impact starts. $c_\tau$ is an experimental factor, apparently around 0.8 for steel.

Using (64), this results in their expression for the force:

$f_n = k_0 \cdot \rho^n \left[1 + \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{\dot\rho}{\dot\rho^{(-)}}\right]$

with $k_0 = \frac{4}{3\cdot(\sigma_1 + \sigma_2)} \cdot \sqrt{\frac{R_1 \cdot R_2}{R_1 + R_2}}$, where $\sigma_i = \frac{1 - \nu_i^2}{E_i}$, with $\nu_i$ = Poisson's ratio, $E_i$ = Young"s modulus, $R_1, R_2$ the radii of the colliding bodies, $\rho$ the penetration depth. All is near equations (54) and (61) of this article.

As per the article, $n = \frac{3}{3}$ is always to be used.

*spring energy* =   $ k_0 \cdot \int_{0}^{\rho} k^{3/2}\,dk$ = $k_0 \cdot\frac{2}{5} \cdot \rho^{5/2}$\
I assume, the dissipated energy cannot be given in closed form, at least the article does not give one.

**Notes**  
1.\
$c_\tau = 1.$ gives **Hertz's** solution to the impact problem, also described in the article\
2.\
For $n > 2$ only $c_\tau = 1.$ gives satisfactory results. I do not know, why.

**Variables**

- $n$: number of pendulums
- $q_1...q_n$: generalized coordinate for the pendulums
- $u_1...u_n$: the angular speeds
- $N$: frame of inertia
- $A_i$: body fixed frame of the i-th pendulum

- $m_i$: mass of the i-th pendulum
- $r_0, l_0, k_0$: radius of pendulum, distance of the center of the pendulum to its suspension point, that is | $\overline{P_{unten_i}P_{oben_i}}$ |, modulus of elasticity of the pendulum body
- $i_{ZZ_i}$: moment of ineratia of the i-th pendulum
- $P_{oben_i}$: suspension point of i-th pendulum
- $P_{unten_i}$: center of mass of i-th pendulum


- $c_\tau$: the experimental constant needed for Hunt-Crossley
- $rhodt_{max}$: the collission speed, to be determined during integration, needed for Hunt_Crossley 

In [ ]:
def HC_force(N, P1, P2, r, ctau, rhodtmax, k0):
    '''
Calculates the contact force exerted by P1 on P2, according to the Hunt-Crossley formula given above.
I assume, that the contact force always acts along the line P1/P2. I think, this is a fair assymption if
the colliding balls are homogenious.

The variables in the list are

- N is an me.ReferenceFrame, the inertial frame
- P1, P2 are me.Point objects. They are the centers of two me.RigidBody objects, here assumed to be two
  ball each of radius r
- radius of the ball
- ctau, the experimental constant needed
- rhodtmax, the relative speeds of P1 to P2, right at the impact time, measured in N. 
  This has to be calculated numerically during the integration, I found no ther way.
- k0 is the force constant

When I take the magnitude of the penetration speed, called geschw, I 'loose' its direction. 
I use vorzeichen to get it.

    '''
    vektorP1P2 = P2.pos_from(P1)
    abstand = vektorP1P2.magnitude() 
    richtung = vektorP1P2.normalize()        
    
    rho = 2. * r - abstand       # penetration. Positive if the two balls are in collision
    geschw = vektorP1P2.diff(t, N)
    vorzeichen = sm.sign(me.dot(geschw, vektorP1P2))
    rhodt = vorzeichen * geschw.magnitude()
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft = k0 * rho**(3/2) * (1. + 3./2. * (1 - ctau) * rhodt/rhodtmax
        ) * sm.Heaviside(2. * r - abstand, 0.) * richtung
    
    return kraft   

In [ ]:
start = time.time()
#-----------------------------------------------------------------------------------------------------------
n = 2                      # number of pendulums, must the >=2
#------------------------------------------------------------------------------------------------------------------------------------
N = me.ReferenceFrame('N')
P0 = me.Point('P0')
P0.set_vel(N, 0)

q = []
u = []

masse = []                                      # different particles may have different masses.
inertia = []
A = []
for i in range(n):                              # Note: dates for point i are on place i-1
    q.append(me.dynamicsymbols('q'+str(i), real=True))     # Rotation of frame i relative to N
    u.append(me.dynamicsymbols('u'+str(i), real=True))     # angular velocity 
    masse.append(sm.symbols('m' + str(i)))
    inertia.append(sm.symbols('iZZ' + str(i)))
    A.append(me.ReferenceFrame('A' + str(i)))
    
g, r0, l0, k0, ctau = sm.symbols('g, r0, l0, k0, ctau', real=True)    
t = sm.symbols('t')

P_oben = []                # attachment point of pendulum on top
P_unten = []               # pendulum point
BODY = []
traegheit = []             # this holds the moments of inertia of the pemdulum bodies
rhodtmax = []              # relative speeds P_i / P_i+1 right at the impact

for i in range(n):
    A[i].orient_axis(N, q[i], N.z)
    A[i].set_ang_vel(N, u[i]*N.z)
    P_oben.append(P0.locatenew('P0' + str(i), 2*i*r0*N.x))
    P_unten.append(P_oben[i].locatenew('P1' + str(i), l0*A[i].y))
    traegheit.append(me.inertia(A[i], 0., 0., inertia[i]))
    rhodtmax.append(sm.symbols('rhodtmax' + str(i)))
    
    BODY.append( (me.RigidBody('P1'+str(i)+'a', P_unten[i], A[i], masse[i], (traegheit[i], P_unten[i]))) )

#Kinemat. equations
kd = [*[u[i] - q[i].diff(t) for i in range(n)]]

# Impact forces. Leftmost and rightmost pendulae are different from the ones in the middle
FL = [(P_unten[0], HC_force(N, P_unten[1], P_unten[0], r0, ctau, rhodtmax[0], k0))]
for i in range(1, n-1):
    print('da')
    FL.append((P_unten[i], HC_force(N, P_unten[i-1], P_unten[i], r0, ctau, rhodtmax[i-1], k0) + 
               HC_force(N, P_unten[i+1], P_unten[i], r0, ctau, rhodtmax[i], k0)  ))
FL.append((P_unten[n-1], HC_force(N, P_unten[n-2], P_unten[n-1], r0, ctau, rhodtmax[n-2], k0)))

# gravitational forces
for i in range(n):
    FL.append((P_unten[i], -masse[i] * g * N.y))

# Get the speed of P_i / P_i+1- needed for the HC method.
rhodt_impact = []
distanz1 = []
for i in range(n-1):
    abstand = P_unten[i+1].pos_from(P_unten[i])
    distanz1.append(abstand.magnitude())
    geschw = (abstand.diff(t, N)).subs({sm.Derivative(l, t): j for l, j in zip(q, u)})
    vorzeichen = sm.sign(me.dot(abstand, geschw))
    rhodt_impact.append(vorzeichen * geschw.magnitude())

#Kane's equations
KM = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM contains {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0])
        for j in range(MM.shape[1])])), '\n')

force = KM.forcing_full
print('force DS', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print('force contains {} operations'.format(sum([force[i].count_ops(visual=False) 
        for i in range(force.shape[0]) ])), '\n')

P_unten_list = [[me.dot(P_unten[i].pos_from(P0), uv) for uv in (N.x, N.y)] for i in range(n)]
P_oben_list = [[me.dot(P_oben[i].pos_from(P0), uv) for uv in (N.x, N.y)] for i in range(n)]

# Always good to look at the energy of the system
pot_energie = sum([masse[i] * g * me.dot(P_unten[i].pos_from(P0), N.y) for i in range(n)])
kin_energie = sum([BODY[i].kinetic_energy(N) for i in range(n)])


# I must use the sm.Max(, 0) method, as a**(5/2) gives problems when a < 0.
spring_energie = 0.
for i in range(n-1):
    deltas = sm.Max(2.*r0 - distanz1[i], sm.S(0.))
    deltas = deltas**(5/2)
    spring_energie += k0 * 2./5. * deltas * sm.Heaviside(2.*r0 - distanz1[i], 0.)

*Force during penetration*\
This is needed only for the plotting of the *hysteresis curves* below

In [ ]:
vektorP1P2 = P_unten[1].pos_from(P_unten[0])
abstand = vektorP1P2.magnitude() 
richtung = vektorP1P2.normalize()        

geschw = vektorP1P2.diff(t, N)
vorzeichen = sm.sign(me.dot(vektorP1P2, geschw))
rho = 2. * r0 - abstand       # penetration. Positive if the two balls are in collision
rhodt = vorzeichen * geschw.magnitude()
rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
kraft = (k0 * rho**(3/2) * (1. + 3./2. * (1 - ctau) * rhodt/rhodtmax[0]
        ) * sm.Heaviside(2. * r0 - abstand, 0.)).subs({sm.Derivative(q[0], t): u[0],
                sm.Derivative(q[1], t): u[1]})
print('kraft dynamic symbols', me.find_dynamicsymbols(kraft, reference_frame=N))
print('kraft free symbols   ', kraft.free_symbols)

Here the *sympy functions* are converted to *numpy functions* so numerical calculations may be done. 

In [ ]:
#Lambdification

#def myHeaviside(x):             
#needed so lambdify knows what Heaviside is. (Trick from Jason Moore) Not
# needed anymore as Heaviside has been implemented into sympy.
# I do not delete it, so I know how to do it if some other function is not implemented in sympy.
#    return np.heaviside(x, 0.5)

qL = q + u
pL = masse + inertia + [g, r0, l0, k0, ctau] + rhodtmax
pLL = masse + inertia + [g, r0, l0, k0, ctau]

MM_lam = sm.lambdify(qL + pL, MM, cse=True) #,  [{'Heaviside': myHeaviside}, 'numpy'] )  # Das 'numpy' ist notwendig.
force_lam = sm.lambdify(qL + pL, force, cse=True) #, [{'Heaviside': myHeaviside}, 'numpy'] )

P_oben_lam = sm.lambdify(qL + pL, P_oben_list, cse=True)
P_unten_lam = sm.lambdify(qL + pL, P_unten_list, cse=True)

pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)
kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)
spring_lam = sm.lambdify(qL + pL, spring_energie, cse=True)

rhodt_impact_lam = sm.lambdify(qL + pLL, rhodt_impact, cse=True)
distanz1_lam = sm.lambdify(qL + pLL, distanz1, cse=True)

kraft_lam = sm.lambdify(qL + pL, kraft, cse=True)

print('it took {:.3f} sec sec to establish Kanes equations'.format(time.time() - start))

**Numerical Integration**

All is standard.\
In *solve_ivp* using *method = 'BDF'* seems to be better here than using *method = 'Radau'*.\
*rhodtmax* is found numerically during the integration. I believe, this is the most 'tricky' part of the whole program.

The number of result points given by solve_ivp, this value is given by *schritte* must be very large to get a nice plot of the *hysteresis curves*\
If I use the Young modulus for steel, around $E_Y = 2. \cdot 10^{11} \frac{N}{m^2}$, I would mave to make max_steps still much smaller, resulting in very long integration times.

In [ ]:
#Numerical integration
start1 = time.time()

# Input variables
#-------------------------------------------------------------------
r01 = 1.0           # radius of the pendulum
l01 = 10.           # distance of particles from the ceiling
m01 = 1.            # mass of a pendulum
ctau1 = 0.8         # given in the article

intervall = 20                          

q01 = [np.pi for i in range(n)] # all particles hang down verticvally, except the first one
u01 = [0. for i in range(n)]    # all have angular velocity = 0. 
q01[0] = 0.5 * np.pi            # first particle moved
#-------------------------------------------------------------------
if n > 2:
    ctau1 = 1.       # for n > 2, only this case works well
    
max_step = 0.001

rhodtmax1 = [1. + k for k in range(len(rhodtmax))] # of no importance, could be anything

nu = 0.28                       # Poisson's ratio, from the internet
EY = 2.e7                       # units: N/m^2, Young's modulus from the internet, around 2e11 for steel
sigma = (1 - nu**2) / EY
k01 = 4. / (3.* (sigma + sigma)) * np.sqrt(r01**2/(r01 + r01))
print(f'k01 = {k01:.3e}')

iZZa = 0.25 * m01 * r01**2      # from the internet

schritte = 300000     
times = np.linspace(0, intervall, schritte)
y0 = q01 + u01

pL_vals  = [m01 for i in range(n)] + [iZZa for i in range(n)] + [9.8, r01, l01, k01, ctau1] + rhodtmax1

def gradient(t, y, args):
    
# here I find rhodtmax, the speed just before the impact
    for i in range(n-1):
        hilfss = distanz1_lam(*y, *[args[k] for k in range(2*n+5)])[i] - 2.*args[2*n+1]
        if (0. < hilfss < 0.001 and y[n+1+i] != 0.) or (0 < hilfss < 0.001 and y[n+i] != 0.):
            args[2 * n + 5 + i] = rhodt_impact_lam(*y, *[args[k] for k in range(int(2*n+5))])[i]
                    
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

    
resultat1 = solve_ivp(gradient, (0., float(intervall)), y0, args=(pL_vals,), t_eval=times, method='BDF', 
            max_step=max_step, atol=1.e-8, rtol=1.e-8)
resultat = resultat1.y.T
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status])

print('to calculate an intervall of {:.0f} sec it took {} loops and {:.3f} sec running time'.
      format(intervall, resultat1.nfev, time.time() - start1))
#==================================================================================

*Plot the coordinates*\
As the number of data points is very large, much larger than needed here, I reduce them to $N_2$ to speed up plotting

In [ ]:
N2 = 5000
N1 = int(resultat.shape[0] / N2)
times1 = []
resultat1 = []
for i in range(resultat.shape[0]):
    if i % N1 == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])
resultat1 = np.array(resultat1)
times1 = np.array(times1)

# print generalized coordinates
fig, ax = plt.subplots(figsize=(10, 5))
for i in range(2*n):
    ax.plot(times1[: resultat1.shape[0]], resultat1[:, i], label='Koordinate {}'.format(i))
ax.set_title(f'Generalized coordinates and angular velocities, ctau = {ctau1}', fontsize = 12)
ax.legend();
print(pL_vals)

*Plot the energies of the system*

In order to get fairly constant total energy, the values of $a_{tol}, r_{tol}$ must be set ( in solve_ivp ) smaller than standard.\
As the impact times are very short, they do not always show in the plot: they are not returned by *solve_ivp*.\
for $EY \approx 10^7$ the impacts are visible.

As the number of results is very large, much larger than needed here, I reduce it to $N_2$ data points. This speeds up the plotting

In [ ]:
N2 = 20000
N1 = int(resultat.shape[0] / N2)
times1 = []
resultat1 = []
for i in range(resultat.shape[0]):
    if i % N1 == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])
resultat1 = np.array(resultat1)
times1 = np.array(times1)

# print energies
pot_np = np.empty(resultat1.shape[0])
kin_np = np.empty(resultat1.shape[0])
spring_np = np.empty(resultat1.shape[0])
total_np = np.empty(resultat1.shape[0])


for i in range(resultat1.shape[0]):
    pot_np[i] = pot_lam(*[resultat1[i, j]  for j in range(resultat1.shape[1])], *pL_vals)
    kin_np[i] = kin_lam(*[resultat1[i, j]  for j in range(resultat1.shape[1])], *pL_vals)
    spring_np[i] = spring_lam(*[resultat1[i, j]  for j in range(resultat1.shape[1])], *pL_vals)
    total_np[i] = pot_np[i] + kin_np[i] + spring_np[i]

min_total = np.abs(np.min(total_np))
max_total = np.abs(np.max(total_np))

print('Max. deviation of total energy from being constant is {:.2e} % of max total energy'.
    format((max_total - min_total)/max_total * 100.))

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times1[: resultat1.shape[0]], pot_np, label='potential energy')
ax.plot(times1[: resultat1.shape[0]], kin_np, label='kinetic energy')
ax.plot(times1[: resultat1.shape[0]], spring_np, label='spring energy')
ax.plot(times1[: resultat1.shape[0]], total_np, label='total energy')
ax.set_title('Energies of the system of {} bodies, ctau = {}'.format(n, ctau1), fontsize=12)
ax.legend();

**Hysteresis curve**\
Here the hysteresis curves of the first ball are plotted.\
They get 'smaller' at subsequet impacts, as is to be expected: Energy is lost at each impact.

In [ ]:
HC_kraft = []
HC_displ = []

for i in range(resultat.shape[0]):
    if distanz1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *[pL_vals[k]
            for k in range(2*n+5)])[0] - 2.*pL_vals[2*n+1] <= 0.:
        HC_displ.append(-(distanz1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
            *[pL_vals[k] for k in range(2*n+5)])[0] - 2.*pL_vals[2*n+1]))
        HC_kraft.append(kraft_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals))

HC_displ = np.array(HC_displ)
HC_kraft = np.array(HC_kraft)

fig, ax = plt.subplots(figsize=(10,5))
ax.plot(HC_displ, HC_kraft)
ax.set_xlabel('penetration depth (m)')
ax.set_ylabel('contact force (Nm)')
ax.set_title('hysteresis curves of successive impacts, ctau = {}'.format(ctau1));

*Animation*

I limit the number of time points to be considered to around *zeitpunkte*, do reduce the time it takes to plot. HTML is VERY slow.

In [ ]:
#Animation

times2 = []
resultat2 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2.append(resultat[i])

schritte2 = len(times2)
resultat2 = np.array(resultat2)
times2 = np.array(times2)
print('number of points considered:',len(times2))

Koordinaten_unten = [np.zeros((schritte2, 2))for i in range(n)]
Koordinaten_oben = [np.zeros(2) for i in range(n)]

for i in range(n):
    for j in range(schritte2):
        Koordinaten_unten[i][j, 0] = P_unten_lam(*[resultat2[j, k]     # X coordinate of i_th pendulum
                for k in range(resultat.shape[1])], *pL_vals)[i][0]       
        Koordinaten_unten[i][j, 1] = P_unten_lam(*[resultat2[j, k]     # Y coordinate of i_th pendulum
                for k in range(resultat.shape[1])], *pL_vals)[i][1]
        
    Koordinaten_oben[i][0] = P_oben_lam(*[resultat2[0, k]   
                for k in range(resultat.shape[1])], *pL_vals)[i][0]       
    Koordinaten_oben[i][1] = P_oben_lam(*[resultat2[0, k]   
                for k in range(resultat.shape[1])], *pL_vals)[i][1] 

obenx = np.zeros(n)
obeny = np.zeros(n)
for i in range(n):
    obenx[i] = Koordinaten_oben[i][0]
    obeny[i] = Koordinaten_oben[i][1]

def animate_pendulum(times2, Koordinaten_unten):
    xmin = min([Koordinaten_unten[0][j, 0] for j in range(schritte2)])
    xmax = max([Koordinaten_unten[n-1][j, 0] for j in range(schritte2)])
    
    ymin = min([Koordinaten_unten[0][j, 1] for j in range(schritte2)])
    ymax = max([Koordinaten_unten[n-1][j, 1] for j in range(schritte2)])
    
    kleinst = min(xmin, ymin)
    groesst = max(xmax, ymax)
    
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.set_aspect('equal')
    ax.axis('on')
    ax.set(xlim=(kleinst - 1., groesst + 1.), ylim=(kleinst - 1., 1.))
    ax.scatter(obenx, obeny, marker='o', linewidths=5, color='black') # points on the ceiling
    
    LINE1 = []
    LINE2 = []
    for i in range(n):
# picking the 'right' radius of the discs I do by trial and error. I did not try to get a formula
        line1, = ax.plot([], [], 'o', markersize= 45 + 5.*(8-n))
        line2, = ax.plot([], [], lw=0.5, color='blue')
        LINE1.append(line1)
        LINE2.append(line2)

    def animate(i):
        ax.set_title('Pendulum of {} bodies, running time {:.2f} sec'.format(n, i/schritte2 * intervall)
                     , fontsize=20)
        for j in range(n):
            x = Koordinaten_unten[j][:, 0]
            y = Koordinaten_unten[j][:, 1]
            LINE1[j].set_data(x[i], y[i])
            LINE2[j].set_data((obenx[j], x[i]), (obeny[j], y[i]))
        return LINE1 + LINE2

    anim = animation.FuncAnimation(fig, animate, frames=schritte2,
                                   interval=1000*times2.max() / schritte2,
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times2, Koordinaten_unten)
print(f'it took {time.time() - start :.3f} sec to run the program, BEFORE HTML')
HTML(anim.to_jshtml())